<h1 align="center"><b>KOTAK SALESIAN SCHOOL</b></h1>


<h2 align="center"><b>STUDENTS INFO 2024-25</b></h2>

## **Backup Files Before running New**

In [29]:
import os
import datetime
import subprocess

# * PostgreSQL Credentials
DB_USER = "postgres"
DB_PASSWORD = "Hari@123"
DB_NAME = "schooldb"
DB_HOST = "localhost"
DB_PORT = "5432"
BACKUP_DIR = "D:/postgres_backups"  # * Backup directory

# * Full path to pg_dump (if needed)
PG_DUMP_PATH = r"C:\Program Files\PostgreSQL\17\bin\pg_dump.exe"

# * Ensure the backup directory exists
os.makedirs(BACKUP_DIR, exist_ok=True)

# * Generate a timestamp for the backup file
timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
backup_file = os.path.join(BACKUP_DIR, f"backup_{DB_NAME}_{timestamp}.sql")

# * Run pg_dump
try:
    result = subprocess.run(
        [
            PG_DUMP_PATH,  # Use full path if not in PATH
            "-U", DB_USER,
            "-h", DB_HOST,
            "-p", DB_PORT,
            "-F", "c",
            "-b",
            "-v",
            "-f", backup_file,
            DB_NAME
        ],
        env={**os.environ, "PGPASSWORD": DB_PASSWORD},
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True,
    )

    # * Check for errors
    if result.returncode == 0:
        print(f"✅ Backup successful: {backup_file}")
    else:
        print(f"❌ Backup failed!\nError: {result.stderr}")

except FileNotFoundError:
    print(f"⚠️ pg_dump not found at {PG_DUMP_PATH}. Check PostgreSQL installation or system PATH.")

except Exception as e:
    print(f"⚠️ An unexpected error occurred: {e}")


✅ Backup successful: D:/postgres_backups\backup_schooldb_2025-03-08_18-22-29.sql


## **Import Libraries & Define Credentials**

In [30]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from sqlalchemy import create_engine, text
import urllib.parse

GOOGLE_JSON_PATH = r"D:\GITHUB\kotak-school-dbms\google_api_keys\woven-solution-446513-f2-6700b7a9f290.json"
GOOGLE_SHEET_NAME = "Fee Reports 2024-25"
POSTGRES_CREDENTIALS = {
    "username": "postgres",
    "password": "Hari@123",
    "host": "localhost",
    "port": "5432",
    "database": "schooldb",
}
TABLE_NAME = "students_2024_25"

## **Extract Data from Google Sheet**

In [31]:
def fetch_data(sheet_name="Sheet1"):
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(GOOGLE_JSON_PATH, scope)
    client = gspread.authorize(creds)

    try:
        # * Open the Google Spreadsheet
        spreadsheet = client.open("STUDENTS DETAILS 2024-25")
        
        # * Open the specific sheet (default is Sheet1)
        sheet = spreadsheet.worksheet("Overall")
    except gspread.SpreadsheetNotFound:
        raise Exception("❌ Spreadsheet not found! Ensure the name is correct and the service account has access.")
    except gspread.WorksheetNotFound:
        raise Exception(f"❌ Worksheet '{sheet_name}' not found! Ensure the name matches exactly.")
    
    # * Fetch data
    data = sheet.get_all_records(head=3)
    return pd.DataFrame(data)

In [32]:
# * Fetch data
student_info = fetch_data()

student_info.head()

,S.No,Adm No.,Name,Class,Gender,Mother's Name,Father's Name,Pen Number,D.O.B,Ph. No.,Religion,Caste,Sub Caste,2nd Lang.,Remarks,Class Nos,JOINED YEAR,GRADES
0,1,17159,CHAITHRA PRESINGU,Pre KG,Female,,,,13-12-2021,7093316614,Hindu,BC - A,,Telugu,,1,,1
1,2,16935,DHRUVITA PRAARTHANA SAVARAPU,Pre KG,Female,,,,18-07-2021,9912246176,Hindu,SC,JAMBAVULU,Telugu,NO CASTE CERTIFICATE,1,,1
2,3,17158,DHRUVITHA ALPHONSEY SONGA,Pre KG,Female,,,,28-07-2021,8341709383,Hindu,SC,MADIGA,Telugu,NO CASTE CERTIFICATE,1,,1
3,4,17160,HARICHANDANA MARADANA,Pre KG,Female,,,,18-12-2020,8464066346,Hindu,BC - D,,,,1,,1
4,5,16973,JYOTHIRMAE NADIGATLA,Pre KG,Female,,,,11-11-2021,9701308587,Hindu,BC - A,,Telugu,,1,,1


## **Clean Extracted Data**

In [33]:
import pandas as pd

def clean_data(df):
    # * Make a copy to avoid SettingWithCopyWarning
    df = df.copy()

    # * Rename columns
    df.columns = [
        'SNo', 'AdmissionNo', 'STUDENT_NAME', 'Class', 'Gender', 'MotherName', 
        'FatherName', 'PenNo', 'DOB', 'Mobile', 'Religion', 'Caste', 
        'SubCaste', 'IIndLang', 'Remarks', 'ClassNo', 'JoinedYear', "APAAR Status"
    ]
    
    

    # * Keep only valid date formats (DD-MM-YYYY)
    # *df = df[df["DOB"].str.match(r'^\d{1,2}-\d{1,2}-\d{4}$', na=False)]

    # * Convert DOB to MySQL format (YYYY-MM-DD)
    df["DOB"] = pd.to_datetime(df["DOB"], format="%d-%m-%Y", errors='coerce').dt.strftime("%Y-%m-%d")

    # * Sort by ClassNo first, then SNo for logical ordering
    df = df.sort_values(by=["ClassNo", "SNo"], ascending=[True, True])

    # * Drop 'APAAR Status' column if it exists
    if "APAAR Status" in df.columns:
        df = df.drop(columns=["APAAR Status"])

    # * Reset SNo after sorting
    df["SNo"] = range(1, len(df) + 1)

    # * Convert 'JoinedYear' safely to integer (handling empty values)
    df["JoinedYear"] = pd.to_numeric(df["JoinedYear"], errors="coerce").astype("Int64")
    
    df.columns = df.columns.str.lower()
        
    df.to_csv(r"D:\GITHUB\kotak-school-dbms\output_data\students_table_2024_25.csv", index=False)

    return df


## **Update MySQL Database**

In [34]:
def update_database(df):
    password = urllib.parse.quote(POSTGRES_CREDENTIALS["password"])
    engine = create_engine(f"postgresql+psycopg2://{POSTGRES_CREDENTIALS['username']}:{password}"
                           f"@{POSTGRES_CREDENTIALS['host']}:{POSTGRES_CREDENTIALS['port']}/{POSTGRES_CREDENTIALS['database']}")

    try:
        with engine.connect() as conn:
            # Enable autocommit
            conn.execute(text("COMMIT;"))

            # * Step 1: Truncate the table (Clearing old data)
            conn.execute(text(f"TRUNCATE TABLE {TABLE_NAME} RESTART IDENTITY CASCADE;"))
            print(f"✅ All records from the '{TABLE_NAME}' table have been deleted.\n")

            # * Step 2: Insert data into the table
            df.to_sql(name=TABLE_NAME, con=engine, if_exists='append', index=False, method='multi')
            print(f"✅ Data successfully inserted into the '{TABLE_NAME}' table.\n")

            # * Step 3: Verify the inserted data
            result = conn.execute(text(f"SELECT COUNT(*) FROM {TABLE_NAME};"))
            count = result.scalar()
            print(f"✅ {count} records now exist in the '{TABLE_NAME}' table.\n")

    except Exception as e:
        print(f"❌ An error occurred: {e}")


## **Run the Main Function**

In [35]:
if __name__ == "__main__":
    
    # * Fetch data
    student_info = fetch_data()
    print("✅ Data fetched successfully.\n")

    # * Clean data
    student_info = clean_data(student_info)
    print("✅ Data cleaned successfully.\n")
    print("✅ Columns are:\n",student_info.columns)
    print("✅ Data preview before inserting into database:")
    print(student_info.head())  # Show first few rows
    print(f"Total records in DataFrame: {len(student_info)}")


    # * Update database
    update_database(student_info)
    print("✅ Process completed successfully.\n")

    # * Print the full DataFrame
    # *print(student_info.to_string())  # * Print the entire DataFrame in a readable format


✅ Data fetched successfully.

✅ Data cleaned successfully.

✅ Columns are:
 Index(['sno', 'admissionno', 'student_name', 'class', 'gender', 'mothername',
       'fathername', 'penno', 'dob', 'mobile', 'religion', 'caste', 'subcaste',
       'iindlang', 'remarks', 'classno', 'joinedyear'],
      dtype='object')
✅ Data preview before inserting into database:
   sno admissionno                  student_name   class  gender mothername  \
0    1       17159             CHAITHRA PRESINGU  Pre KG  Female              
1    2       16935  DHRUVITA PRAARTHANA SAVARAPU  Pre KG  Female              
2    3       17158     DHRUVITHA ALPHONSEY SONGA  Pre KG  Female              
3    4       17160         HARICHANDANA MARADANA  Pre KG  Female              
4    5       16973          JYOTHIRMAE NADIGATLA  Pre KG  Female              

  fathername penno         dob      mobile religion   caste   subcaste  \
0                   2021-12-13  7093316614    Hindu  BC - A              
1                 

<h2 align="center"><b>FEE REPORT 2024-25</b></h2>

## **Google Console Service Account: myschooldb@woven-solution-446513-f2.iam.gserviceaccount.com**

## **Import Necessary Libraries & Define Global Variables**

In [37]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from sqlalchemy import create_engine, text

GOOGLE_JSON_PATH = r"D:\GITHUB\kotak-school-dbms\google_api_keys\woven-solution-446513-f2-5ffd100e19c7.json"
GOOGLE_SHEET_NAME = "Fee Reports 2024-25"
POSTGRES_CREDENTIALS = {
    "username": "postgres",
    "password": "Hari@123",
    "host": "localhost",
    "port": "5432",
    "database": "schooldb",
}
TABLE_NAME = "fees_table_2024_25"


## **Function for Fetching Data**

In [38]:
def fetch_data(sheet_name="Sheet1"):
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(GOOGLE_JSON_PATH, scope)
    client = gspread.authorize(creds)

    try:
        # * Open the Google Spreadsheet
        spreadsheet = client.open("Fee Reports 2024-25")
        
        # * Open the specific sheet (default is Sheet1)
        sheet = spreadsheet.worksheet("Overall Sheet")
    except gspread.SpreadsheetNotFound:
        raise Exception("❌ Spreadsheet not found! Ensure the name is correct and the service account has access.")
    except gspread.WorksheetNotFound:
        raise Exception(f"❌ Worksheet '{sheet_name}' not found! Ensure the name matches exactly.")
    
    # * Fetch data
    data = sheet.get_all_records(head=3)
    return pd.DataFrame(data)


## **Function for Cleaning Data**

In [39]:
def clean_data(df):
    df = df[:-1][:-6]
    df.columns = ['SNo', 'STUDENT_NAME', 'ADM_NO', 'FB_NO', 'CLASS',
                  'Term1', 'Term2', 'Term3', 'Term4', 'TotalFeePaid',
                  'Discount_Concession', 'TotalFeeDue', 'PermissionUpto',
                  'Fine', 'PaymentStatus', 'ClassNo',"AcNo",'Concession_type', "dummy"]

    columns_to_convert = ["Term1", "Term2", "Term3", "Term4", "TotalFeePaid",
                          "Discount_Concession", "TotalFeeDue", "Fine"]
    df[columns_to_convert] = df[columns_to_convert].apply(pd.to_numeric, errors='coerce').fillna(0)

    df = df.drop(columns=["AcNo", 'Concession_type', "dummy"])

    df["SNo"] = range(1, len(df) + 1)

    df = df.sort_values(by=["SNo"])

    df["TotalFees"] = df["TotalFeePaid"] + df["Discount_Concession"] + df["TotalFeeDue"]
    
    df.to_csv(r"D:\GITHUB\kotak-school-dbms\output_data\fees_report_2024_25.csv", index=False)
    
    PaymentStatus = df[["PaymentStatus"]].drop_duplicates().reset_index(drop=True)
    PaymentStatus["PaymentStatusId"] = range(1, len(PaymentStatus) + 1)
    PaymentStatus = PaymentStatus[["PaymentStatusId", "PaymentStatus"]]
    PaymentStatus.to_csv(r"D:\GITHUB\kotak-school-dbms\output_data\payment_status_table_2024_25.csv", index=False)
    print("✅ Payment Status Table created successfully.\n")
    
    df = pd.merge(df, PaymentStatus, on="PaymentStatus", how="left")
    
    df.drop(columns=["PermissionUpto","PaymentStatus"], inplace=True)
    
    df.columns = df.columns.str.lower().str.strip() # Lowercase all columns
    

    return df


## **Function for Updating the Database**

In [40]:
from sqlalchemy import create_engine, text
import urllib.parse

# PostgreSQL Connection Credentials
POSTGRES_CREDENTIALS = {
    "username": "postgres",
    "password": "Hari@123",
    "host": "localhost",
    "port": "5432",
    "database": "schooldb",
}

# Encode password for URL safety
password = urllib.parse.quote(POSTGRES_CREDENTIALS["password"])

# Create Engine
engine = create_engine(f"postgresql+psycopg2://{POSTGRES_CREDENTIALS['username']}:{password}"
                       f"@{POSTGRES_CREDENTIALS['host']}:{POSTGRES_CREDENTIALS['port']}/{POSTGRES_CREDENTIALS['database']}")

def table_exists(table_name):
    """Check if table exists in PostgreSQL database"""
    check_query = f"""
    SELECT EXISTS (
        SELECT FROM information_schema.tables 
        WHERE table_name = '{table_name}'
    );
    """
    with engine.connect() as conn:
        result = conn.execute(text(check_query)).scalar()
    return result

def create_table():
    """Create table only if it does not exist"""
    table_name = "fees_table_2024_25"
    
    if table_exists(table_name):
        print(f"✅ Table '{table_name}' already exists.")
        return

    create_table_query = """
    CREATE TABLE fees_table_2024_25 (
        SNo SERIAL PRIMARY KEY,
        STUDENT_NAME VARCHAR(100),
        ADM_NO VARCHAR(20) UNIQUE NOT NULL,
        FB_NO VARCHAR(20),
        CLASS VARCHAR(20),
        Term1 NUMERIC(10,2) DEFAULT 0,
        Term2 NUMERIC(10,2) DEFAULT 0,
        Term3 NUMERIC(10,2) DEFAULT 0,
        Term4 NUMERIC(10,2) DEFAULT 0,
        TotalFeePaid NUMERIC(10,2) DEFAULT 0,
        Discount_Concession NUMERIC(10,2) DEFAULT 0,
        TotalFeeDue NUMERIC(10,2) DEFAULT 0,
        Fine NUMERIC(10,2) DEFAULT 0,
        ClassNo INT,
        TotalFees NUMERIC(10,2) GENERATED ALWAYS AS 
            (TotalFeePaid + Discount_Concession + TotalFeeDue) STORED,
        PaymentStatusId INT
    );
    """
    
    try:
        with engine.connect() as conn:
            conn.execute(text(create_table_query))
            print(f"✅ Table '{table_name}' created successfully.")
    except Exception as e:
        print(f"❌ Error creating table: {e}")


In [41]:
from sqlalchemy.exc import SQLAlchemyError

def update_database(df):
    password = urllib.parse.quote(POSTGRES_CREDENTIALS["password"])
    engine = create_engine(f"postgresql+psycopg2://{POSTGRES_CREDENTIALS['username']}:{password}"
                           f"@{POSTGRES_CREDENTIALS['host']}:{POSTGRES_CREDENTIALS['port']}/{POSTGRES_CREDENTIALS['database']}")

    try:
        with engine.begin() as conn:
            # Truncate table and reset identity
            conn.execute(text(f"TRUNCATE TABLE {TABLE_NAME} RESTART IDENTITY CASCADE;"))
            print(f"✅ All records from the '{TABLE_NAME}' table have been deleted.")

            # Ensure unique constraint exists before adding
            conn.execute(text(f"""
                DO $$ 
                BEGIN 
                    IF NOT EXISTS (
                        SELECT 1 FROM information_schema.table_constraints 
                        WHERE table_name = '{TABLE_NAME}' AND constraint_name = 'unique_adm_no'
                    ) THEN
                        ALTER TABLE {TABLE_NAME} ADD CONSTRAINT unique_adm_no UNIQUE ("adm_no");
                    END IF;
                END $$;
            """))
            print(f"✅ Unique constraint on 'adm_no' ensured in the '{TABLE_NAME}' table.")

        print("✅ Table cleared. Proceeding with data insertion...\n")

        # Convert column names to lowercase to match PostgreSQL
        df.columns = df.columns.str.lower()

        # Drop `totalfees` since it's a generated column
        if "totalfees" in df.columns:
            df = df.drop(columns=["totalfees"])
            print("✅ 'totalfees' column removed before insertion (generated column).")

        # Insert data in chunks for efficiency
        df.to_sql(name=TABLE_NAME, con=engine, if_exists='append', index=False, method='multi', chunksize=1000)

        print(f"✅ Data successfully inserted into the '{TABLE_NAME}' table.")

    except SQLAlchemyError as e:
        print(f"❌ An error occurred: {e}")


## **Main Execution Block**

In [42]:
if __name__ == "__main__":
    # * Fetch data
    fees_df = fetch_data()
    print("✅ Data fetched successfully.\n")
    print(fees_df.to_string())

    # * Clean data
    fees_df = clean_data(fees_df)
    print("✅ Data cleaned and transformed successfully.\n")
    print("✅ Columns are:\n", fees_df.columns)
    
    # Run the function to check and create the table
    create_table()

    print("\n✅ Table check complete. Proceeding with database update...\n")

    # * Update database
    update_database(fees_df)

    print("\n✅ Database update completed successfully.")


✅ Data fetched successfully.

     S.No                                       STUDENT NAME  ADM NO       FB NO     CLASS          1st Term JUN - AUG 2nd Term SEP - NOV 3rd Term DEC - FEB    4th Term MAR - MAY Total \nFee Paid Discount / Concession Total Fee \nDue Permission upto   Fine Payment Status Class No      AC No Concession_type        STAFF NAME
0       1                                  CHAITHRA PRESINGU   17159        3001    Pre KG  NO FEES FOR 1ST & 2ND TERM                                  5250                  5250            10500                                     0                            Total Paid        1  A/C No. 1                                  
1       2                       DHRUVITA PRAARTHANA SAVARAPU   16935        3002    Pre KG                                                              5250                  5250            10500                                     0                            Total Paid        1  A/C No. 1                           

In [43]:
fees_df

,sno,student_name,adm_no,fb_no,class,term1,term2,term3,term4,totalfeepaid,discount_concession,totalfeedue,fine,classno,totalfees,paymentstatusid
0,1,CHAITHRA PRESINGU,17159,3001,Pre KG,0.0,0.0,5250.0,5250.0,10500,0.0,0,0.0,1,10500.0,1
1,2,DHRUVITA PRAARTHANA SAVARAPU,16935,3002,Pre KG,0.0,0.0,5250.0,5250.0,10500,0.0,0,0.0,1,10500.0,1
2,3,DHRUVITHA ALPHONSEY SONGA,17158,3003,Pre KG,0.0,0.0,0.0,0.0,0,0.0,10500,0.0,1,10500.0,2
3,4,HARI CHANDANA MARDANA,17160,3004,Pre KG,0.0,0.0,5250.0,5250.0,10500,0.0,0,0.0,1,10500.0,1
4,5,JYOTHIRMAE NADIGATLA,16973,3005,Pre KG,0.0,0.0,5250.0,0.0,5250,0.0,5250,0.0,1,10500.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1728,1729,SHANMUKH SALAPU,15872,1812,X - C,10450.0,10450.0,10450.0,10450.0,41800,0.0,0,0.0,43,41800.0,1
1729,1730,SIVA HARSHA VELAMAREDDI,15741,1807,X - C,10450.0,10450.0,10450.0,10450.0,41800,0.0,0,0.0,43,41800.0,1
1730,1731,UPENDRA MUTCHI,13863,1822,X - C,10450.0,10450.0,10450.0,0.0,31350,0.0,10450,100.0,43,41800.0,4
1731,1732,VARUN PRESINGI,14360,1832,X - C,10450.0,10450.0,10450.0,0.0,31350,0.0,10450,100.0,43,41800.0,4


<h2 align="center"><b>DAY WISE REPORTS 2024-25</b></h2>

## **Import Required Libraries**

In [45]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib.parse
from sqlalchemy import create_engine, text

## **Define Login Credentials and MySQL Credentials**

In [46]:
# * 🔹 Login Credentials
login_url = "https://app.myskoolcom.tech/kotak_vizag/login"
data_url = "https://app.myskoolcom.tech/kotak_vizag/office_fee/fee_reports_day_wise_receipt_wise_print"

credentials = {
    "uname": "harikiran",
    "psw": "812551"
}

POSTGRES_CREDENTIALS = {
    "username": "postgres",
    "password": "Hari@123",
    "host": "localhost",
    "port": "5432",
    "database": "schooldb",
}

TABLE_NAME = "daywise_fees_collection_2024_25"

## **Define Functions for Each Step**

In [47]:
def login_to_website():
    session = requests.Session()
    login_response = session.post(login_url, data=credentials)

    

    if "Invalid" in login_response.text:
        print("❌ Login failed! Check credentials.\n")
        return None
    else:
        print("✅ Login successful!\n")
        return session


## **Function to Fetch Fee Report Page**

In [48]:
def fetch_fee_report_page(session):
    response = session.get(data_url)
    soup = BeautifulSoup(response.text, "html.parser")
    table = soup.find("table")
    return table

## **Function to Extract Data from Table**

In [49]:
def extract_data_from_table(table):
    rows = []
    for tr in table.find_all("tr"):
        cols = [td.text.strip() for td in tr.find_all("td")]
        if cols:
            rows.append(cols)
    
    header_row = [
        "SNo", "RecieptNo", "Class", "AdmissionNo", "StudentName", 
        "Date", "-", "Abacus / Vediic Maths", "TERM FEE", 
        "ReceivedAmount", "Remarks"
    ]
    
    df = pd.DataFrame(rows, columns=header_row)
    return df


## **Function to Clean Data**

In [50]:
def clean_data(df):
    # * Convert 'Date' column to proper datetime format
    df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y', errors='coerce')

    # * Ensure 'AdmissionNo' is treated as a string (no conversion to numeric)
    df['AdmissionNo'] = df['AdmissionNo'].astype(str)

    # * Find the index where "TERM" first appears in the "SNo" column
    term_index = df[df["SNo"].str.contains("TERM", na=False)].index
    
    df["Class"] = df["Class"].str.replace("/", " - ")
    
    df["ReceivedAmount"] = pd.to_numeric(df["ReceivedAmount"], errors="coerce")
    print(df.info())  # Verify the column is now int64 or float64


    if not term_index.empty:
        # * Drop all rows from the first occurrence of "TERM" onward
        df = df.iloc[:term_index[0]]

        # * Drop unnecessary columns
        df = df.drop(columns=["-", "Abacus / Vediic Maths", "TERM FEE"])


    return df


## **Function to Update Database**

In [51]:
def update_database(df):
    password = urllib.parse.quote(POSTGRES_CREDENTIALS["password"])
    engine = create_engine(f"postgresql+psycopg2://{POSTGRES_CREDENTIALS['username']}:{password}"
                           f"@{POSTGRES_CREDENTIALS['host']}:{POSTGRES_CREDENTIALS['port']}/{POSTGRES_CREDENTIALS['database']}")

    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

    try:
        with engine.connect() as conn:
            # ✅ Run TRUNCATE in a separate transaction
            conn.execute(text("BEGIN;"))  # Start a transaction
            conn.execute(text(f"TRUNCATE TABLE {TABLE_NAME};"))
            conn.execute(text("COMMIT;"))  # Commit immediately
            print(f"✅ Table '{TABLE_NAME}' truncated successfully.\n")

            # ✅ Insert Data (Batch Insert)
            df.to_sql(name=TABLE_NAME, con=engine, if_exists='append', index=False, method="multi", chunksize=1000)
            print(f"✅ {len(df)} records inserted into '{TABLE_NAME}' successfully.\n")

    except Exception as e:
        print(f"⚠️ Error inserting data: {e}")


## **Main Execution Flow**

In [52]:
def main():
    # * Log in to the website
    session = login_to_website()
    if session is None:
        return

    # * Fetch the fee report page
    table = fetch_fee_report_page(session)

    if table:
        print("✅ Table found! Extracting data...\n")

        # * Extract data from the table
        df = extract_data_from_table(table)

        # * Clean the data
        df = clean_data(df)

        # * Save to CSV (optional)
        df.to_csv(r"D:\GITHUB\kotak-school-dbms\output_data\daywise_fee_collection_report_2024_25.csv", index=False)
        print("✅ Data saved to fee_collection_report.csv\n")

        # * Insert data into MySQL database
        update_database(df)
        print("✅ Columns are:\n", df.columns)

        # * Print sample data
        print(f"✅ {len(df)} Records Entered into database")

    else:
        print("❌ Table not found! The page structure might have changed.")


## **Run the Main Function**

In [54]:
# * Run the main function
main()


✅ Login successful!

✅ Table found! Extracting data...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5137 entries, 0 to 5136
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   SNo                    5137 non-null   object        
 1   RecieptNo              5137 non-null   object        
 2   Class                  5135 non-null   object        
 3   AdmissionNo            5137 non-null   object        
 4   StudentName            5135 non-null   object        
 5   Date                   4973 non-null   datetime64[ns]
 6   -                      5135 non-null   object        
 7   Abacus / Vediic Maths  4973 non-null   object        
 8   TERM FEE               4973 non-null   object        
 9   ReceivedAmount         4973 non-null   float64       
 10  Remarks                4973 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(9)
memory usage: 441.6+ 

<h2 align="center"><b>ATTENDANCE REPORT 2024-25</b></h2>

## **Import Libraries**

In [56]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
import time
from datetime import datetime

## **Login to Website**

In [57]:
# 🔹 Constants
login_url = "https://app.myskoolcom.tech/kotak_vizag/login"
attendance_url = "https://app.myskoolcom.tech/kotak_vizag/admin/attedance_grid"

credentials = {
    "uname": "harikiran",
    "psw": "812551"
}

# 🔹 Set ChromeDriver path
chromedriver_path = r"G:\chromedriver-win64\chromedriver-win64\chromedriver.exe"

# 🔹 Set the download folder
download_folder = r"D:\GITHUB\kotak-school-dbms\source_data\Attendance Reports"

# 🔹 Set the required filename
final_filename = "AttendanceUptoFebruary_2024_25.csv"
final_filepath = os.path.join(download_folder, final_filename)

# 🔹 Chrome options
chrome_options = webdriver.ChromeOptions()
prefs = {"download.default_directory": download_folder}  # ✅ Set download directory
chrome_options.add_experimental_option("prefs", prefs)

# 🔹 Start WebDriver
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)
wait = WebDriverWait(driver, 10)

In [58]:
# **Step 1: Login to the website**
driver.get(login_url)
wait.until(EC.presence_of_element_located((By.NAME, "uname"))).send_keys(credentials["uname"])
driver.find_element(By.NAME, "psw").send_keys(credentials["psw"])
driver.find_element(By.NAME, "psw").send_keys(Keys.RETURN)

print("✅ Logged in successfully!")

# **Step 2: Go to Attendance Page**
time.sleep(5)  # Wait for login
driver.get(attendance_url)

# **Step 3: Set Date Range**
start_date = "2025-01-01"
end_date = datetime.today().strftime("%Y-%m-%d")

# **Find and set "From Date"**
from_date_input = wait.until(EC.presence_of_element_located((By.ID, "from_attendance_date")))
driver.execute_script("arguments[0].removeAttribute('readonly')", from_date_input)
from_date_input.clear()
from_date_input.send_keys(start_date)

# **Find and set "To Date"**
to_date_input = wait.until(EC.presence_of_element_located((By.ID, "to_attendance_date")))
driver.execute_script("arguments[0].removeAttribute('readonly')", to_date_input)
to_date_input.clear()
to_date_input.send_keys(end_date)

print(f"✅ Date range set: {start_date} to {end_date}")

# **Step 4: Click "Download CSV"**
try:
    download_button = wait.until(EC.element_to_be_clickable((By.ID, "smaplecsv")))  # ✅ Use correct button ID
    download_button.click()
    print("✅ Download initiated!")
except:
    print("❌ Error: Download button not found!")

# **Step 5: Wait for the file to download**
time.sleep(10)  # Allow time for download

# **Step 6: Delete existing file if it already exists**
try:
    if os.path.exists(final_filepath):
        os.remove(final_filepath)
        print(f"🗑️ Deleted existing file: {final_filename}")
except Exception as e:
    print(f"❌ Error deleting existing file: {e}")

# **Step 7: Rename the latest downloaded file**
try:
    # Find the most recently downloaded file
    downloaded_files = sorted(
        [f for f in os.listdir(download_folder) if f.endswith(".csv")],
        key=lambda x: os.path.getctime(os.path.join(download_folder, x)),
        reverse=True,
    )
    
    if downloaded_files:
        latest_file = os.path.join(download_folder, downloaded_files[0])
        os.rename(latest_file, final_filepath)
        print(f"✅ File renamed to: {final_filename}")
    else:
        print("❌ No downloaded CSV file found!")
except Exception as e:
    print(f"❌ Error renaming file: {e}")

# **Step 8: Close browser**
driver.quit()
print("✅ Process completed successfully!")


✅ Logged in successfully!
✅ Date range set: 2025-01-01 to 2025-03-08
✅ Download initiated!
🗑️ Deleted existing file: AttendanceUptoFebruary_2024_25.csv
✅ File renamed to: AttendanceUptoFebruary_2024_25.csv
✅ Process completed successfully!


In [24]:
import sys
import time

# Simulating a download waiting time
wait_time = 15  # Total wait time in seconds

print("⏳ Waiting for download...", end="", flush=True)

for i in range(wait_time):
    time.sleep(1)
    sys.stdout.write(".")
    sys.stdout.flush()


⏳ Waiting for download..................

## **📌 Step 1: Import Libraries**

In [59]:
import pandas as pd
import urllib.parse
from sqlalchemy import create_engine, text
import logging
import numpy as np
import urllib
import logging
import traceback

# * Configure logging
logging.basicConfig(filename=r"D:\GITHUB\kotak-school-dbms\output_data\attendance_report_2024_25.log", level=logging.ERROR, 
                    format="%(asctime)s - %(levelname)s - %(message)s")


## **📌 Step 2: Define MySQL Credentials & Table Name*

In [60]:
# * MySQL Credentials
POSTGRES_CREDENTIALS = {
    "username": "postgres",
    "password": "Hari@123",
    "host": "localhost",
    "port": "5432",
    "database": "schooldb",
}
TABLE_NAME = "attendance_report_2024_25"


# # **📌 Step 3: Load and Clean Data**

In [61]:
## **📌 Step 3: Load and Clean Data (Updated)*
def load_and_clean_data(file1, file2, file3):


    # * Load Data
    df1 = pd.read_csv(file1)
    df2 = pd.read_csv(file2)
    df3 = pd.read_csv(file3)

    # * Standardize column names
    for df in [df1, df2, df3]:
        df.columns = df.columns.str.strip().str.replace('"', '', regex=False)

    # * Merge DataFrames on 'Students Number' using outer join
    df = df1.merge(df2, on='Students Number', how='outer').merge(df3, on='Students Number', how='outer')

    # * Identify and handle duplicate columns
    common_fields = ['Name', 'Class']
    for field in common_fields:
        df[field] = df.pop(f"{field}_x").combine_first(df.pop(f"{field}_y"))

    # * Drop remaining duplicate columns
    drop_columns = [col for col in df.columns if '_x' in col or '_y' in col]
    df = df.drop(columns=drop_columns, errors='ignore')

    # * Rename 'Students Number' to 'AdmissionNo'
    df = df.rename(columns={"Students Number": "AdmissionNo"})

    # * Reorder Columns
    column_order = ['AdmissionNo', 'Name', 'Class'] + [col for col in df.columns if col not in ['AdmissionNo', 'Name', 'Class']]
    df = df[column_order]

    # * Drop Unnecessary Columns
    columns_to_drop = ["Present Days", "Absent Days", "Toral Working Days"]  # * Ensure correct column names
    df = df.drop(columns=[col for col in columns_to_drop if col in df.columns], errors='ignore')


    return df

# **📌 Step 4: Process Attendance Data**

In [62]:
def process_attendance_data(df):

    # * Step 1: Clean 'AdmissionNo'
    df = df[~(df["AdmissionNo"].astype(str) == "786") & ~df["AdmissionNo"].astype(str).str.match(r"^[a-zA-Z]")].copy()

    # * Step 2: Extract Class and Section
    df["Class"] = df["Class"].astype(str).str.replace(r"ICSE \((.*?)\)", r"\1", regex=True)
    
    student_df = pd.read_csv(r"D:\GITHUB\kotak-school-dbms\output_data\fees_report_2024_25.csv")
    
    print("✅ Students Before Merging\n", len(df["AdmissionNo"].unique()))
    
    # * Step 3: Keep only rows where AdmissionNo is in student_df
    df = df[df["AdmissionNo"].isin(student_df["ADM_NO"])]
    
    print("✅ Students After Merging\n", len(df["AdmissionNo"].unique()))
    
    # * Step 3: Unpivot DataFrame
    df_unpivot = pd.melt(df, id_vars=["AdmissionNo", "Name", "Class"], 
                        var_name="Date", value_name="AttendanceStatus")
    
    # * Step 5: Drop rows with 'Holiday' status
    #df_unpivot = df_unpivot[~df_unpivot["AttendanceStatus"].eq("H")].reset_index(drop=True)

    # * Step 4: Convert 'Date' to datetime
    df_unpivot["Date"] = pd.to_datetime(df_unpivot["Date"], format='%d.%m.%Y', errors='coerce')

    # * Step 5: Log invalid 'Date' values
    if df_unpivot["Date"].isna().sum() > 0:
        print("⚠️ Warning: Some Date values were invalid and converted to NaT.")

    # * Step 6: Sorting
    df_unpivot = df_unpivot.sort_values("Date", ascending=False).reset_index(drop=True)

    # * Step 7: Find the first attendance date for each student
    first_attendance_dates = df_unpivot[df_unpivot['AttendanceStatus'].notna()].groupby('AdmissionNo')['Date'].min()

    # * Step 8: Assign 'Not Joined' if the date is before the student's first attendance
    df_unpivot['AttendanceStatus'] = df_unpivot.apply(
        lambda row: "Not Joined" if row['Date'] < first_attendance_dates.get(row['AdmissionNo'], row['Date']) else row['AttendanceStatus'],
        axis=1
    )
    
    # * Step 9: Prioritize Attendance Status
    priority_map = {'P': 2, 'A': 1, 'H': 3, 'Not Joined': 4, 'TC': 5}
    df_unpivot['Priority'] = df_unpivot["AttendanceStatus"].map(priority_map)

    df_unpivot = df_unpivot.sort_values(by=['AdmissionNo', 'Date', 'Priority']) \
                            .drop_duplicates(subset=['AdmissionNo', 'Date'], keep='first') \
                            .drop(columns=['Priority'])
                                
    # * Step 10: Final sorting
    df_unpivot = df_unpivot[['Date', 'AdmissionNo', 'Name', 'Class', 'AttendanceStatus']]
    df_unpivot.sort_values(by=['Date'], ascending=False, inplace=True)
    
    df_unpivot['Class'] = df_unpivot['Class'].str.replace("Pre KG - ", "Pre KG")

    # * Step 11: Replace Attendance Status with meaningful labels
    df_unpivot["AttendanceStatus"] = df_unpivot["AttendanceStatus"].replace({
        'P': "Present", 'A': "Absent", 'H': "Holiday"})

    # * Step 13: Class & Section Mapping
    class_section_mapping = {
    "Pre KG": 1, "LKG - A": 2, "LKG - B": 3, "UKG - A": 4, "UKG - B": 5, "UKG - C": 6,
    "I - A": 7, "I - B": 8, "I - C": 9, "I - D": 10, "II - A": 11, "II - B": 12, "II - C": 13, "II - D": 14,
    "III - A": 15, "III - B": 16, "III - C": 17, "III - D": 18, "IV - A": 19, "IV - B": 20, "IV - C": 21, "IV - D": 22,
    "V - A": 23, "V - B": 24, "V - C": 25, "V - D": 26, "VI - A": 27, "VI - B": 28, "VI - C": 29, "VI - D": 30,
    "VII - A": 31, "VII - B": 32, "VII - C": 33, "VII - D": 34, "VIII - A": 35, "VIII - B": 36, "VIII - C": 37,
    "IX - A": 38, "IX - B": 39, "IX - C": 40, "X - A": 41, "X - B": 42, "X - C": 43}

    df_unpivot['ClassNo'] = df_unpivot['Class'].map(class_section_mapping)

    # * Step 14: Grade Mapping
    grade_mapping = [
        ("Pre KG", 1), ("LKG", 2), ("UKG", 3),
        ("I", 4), ("II", 5), ("III", 6), ("IV", 7), ("V", 8),
        ("VI", 9), ("VII", 10), ("VIII", 11), ("IX", 12), ("X", 13)
    ]

    conditions = [df_unpivot['Class'].str.contains(fr"\b{k}\b", na=False, regex=True) for k, _ in grade_mapping]
    choices = [v for _, v in grade_mapping]
    df_unpivot['classId'] = np.select(conditions, choices, default=0)

    # * Step 15: AttendanceStatus Mapping
    AttendanceStatus_mapping = [("Absent", 1), ("Present", 2), ("Not Joined", 3), ("Holiday", 4)]
    conditions = [df_unpivot['AttendanceStatus'].str.contains(k, na=False) for k, _ in AttendanceStatus_mapping]
    choices = [v for _, v in AttendanceStatus_mapping]
    df_unpivot['AttendanceStatusId'] = np.select(conditions, choices, default=0)

    # * Step 16: Branch Mapping
    branch_mapping = [
        ('Pre KG', 1), ('LKG', 1), ('UKG', 1),
        ('I', 2), ('II', 2), ('III', 2), ('IV', 2), ('V', 2),
        ('VI', 3), ('VII', 3), ('VIII', 3), ('IX', 3), ('X', 3)
    ]

    conditions = [df_unpivot['Class'].str.contains(fr"\b{k}\b", na=False, regex=True) for k, _ in branch_mapping]
    choices = [v for _, v in branch_mapping]
    df_unpivot['branchId'] = np.select(conditions, choices, default=0)

    # * Step 17: Branch Name Mapping
    branch_name_mapping = {1: 'Kindergarten', 2: 'Primary', 3: 'Higher'}
    df_unpivot['branchName'] = df_unpivot['branchId'].map(branch_name_mapping)
    

    grade_mapping_reversed = {
    1: "Pre KG", 2: "LKG", 3: "UKG",
    4: "I", 5: "II", 6: "III", 7: "IV", 8: "V",
    9: "VI", 10: "VII", 11: "VIII", 12: "IX", 13: "X"
}
    

    df_unpivot['className'] = df_unpivot['classId'].map(grade_mapping_reversed)

    # * Step 19: Final DataFrame Cleanup
    class_sec_ids = df_unpivot[["ClassNo", "Class", "classId", "className", "branchId", "branchName",]].drop_duplicates(subset=["ClassNo"])
    class_sec_ids = class_sec_ids.sort_values(by=['ClassNo']).reset_index(drop=True)
    class_sec_ids.to_csv(r"D:\GITHUB\kotak-school-dbms\output_data\class_section_grade_table_2024_25.csv", index=False)

    # * Step 19: Final DataFrame Cleanup
    # * Step 19: Final DataFrame Cleanup
    class_colors = {
        1: "# *FFC0CB",  # * Pre KG - Pink
        2: "# *FFD700",  # * LKG - Gold
        3: "# *FFA07A",  # * UKG - Light Salmon
        4: "# *ADD8E6",  # * I - Light Blue
        5: "# *90EE90",  # * II - Light Green
        6: "# *FFA500",  # * III - Orange
        7: "# *87CEEB",  # * IV - Sky Blue
        8: "# *32CD32",  # * V - Lime Green
        9: "# *6A5ACD",  # * VI - Slate Blue
        10: "# *008080",  # * VII - Teal
        11: "# *4682B4",  # * VIII - Steel Blue
        12: "# *8B0000",  # * IX - Dark Red
        13: "# *4B0082"   # * X - Indigo
    }

    # * Extract unique class IDs and class names
    class_ids = df_unpivot[["classId", "className"]].drop_duplicates(subset=["classId"])
    class_ids = class_ids.sort_values(by=["classId"]).reset_index(drop=True)

    # * Add color column based on classId
    class_ids["color"] = class_ids["classId"].map(class_colors)
    class_ids.to_csv(r"D:\GITHUB\kotak-school-dbms\output_data\class_table_2024_25.csv", index=False)

    # * Step 19: Final DataFrame Cleanup
    branch_ids = df_unpivot[["branchId", "branchName",]].drop_duplicates(subset=["branchId"])
    branch_ids = branch_ids.sort_values(by=['branchId']).reset_index(drop=True)
    branch_ids.to_csv(r"D:\GITHUB\kotak-school-dbms\output_data\branch_table_2024_25.csv", index=False)
    
    AttendanceStatus_table = df_unpivot[['AttendanceStatusId','AttendanceStatus']].drop_duplicates(subset=["AttendanceStatusId"])
    
    AttendanceStatus_table.to_csv(r"D:\GITHUB\kotak-school-dbms\output_data\AttendanceStatus_table_2024_25.csv", index=False)
    
    df_unpivot.to_csv(r"D:\GITHUB\kotak-school-dbms\output_data\attendance_report_2024_25.csv", index=False)    

    df_unpivot = df_unpivot[['Date', 'AdmissionNo', 'ClassNo', 'classId', 'branchId', 'AttendanceStatusId']]
    
    return df_unpivot

## **📌 Step 5: Insert Data into PostgreSQL**

In [63]:
# Create database engine
password = urllib.parse.quote(POSTGRES_CREDENTIALS["password"])
engine = create_engine(
    f"postgresql+psycopg2://{POSTGRES_CREDENTIALS['username']}:{password}"
    f"@{POSTGRES_CREDENTIALS['host']}:{POSTGRES_CREDENTIALS['port']}/{POSTGRES_CREDENTIALS['database']}"
)

def update_database(df):
    """Use PostgreSQL COPY for ultra-fast data insertion."""
    csv_path = "attendance_data.csv"
    
    # Save DataFrame to CSV (PostgreSQL COPY needs a file)
    df.to_csv(csv_path, index=False, header=False)

    try:
        conn = engine.raw_connection()  # ✅ Get raw connection
        cursor = conn.cursor()
        
        print(f"🔄 Truncating table: {TABLE_NAME}")
        cursor.execute(f"TRUNCATE TABLE {TABLE_NAME};")  # ✅ TRUNCATE before inserting data
        conn.commit()

        with open(csv_path, "r") as f:
            cursor.copy_from(f, TABLE_NAME, sep=",")  # ✅ Use COPY command

        conn.commit()  # ✅ Commit transaction
        cursor.close()
        conn.close()  # ✅ Close connection

        print(f"✅ Data copied to '{TABLE_NAME}' using COPY command!")

    except Exception as e:
        print(f"❌ COPY failed: {e}")
        logging.error(traceback.format_exc())

## **📌 Step 6: Run the Full Pipeline**

In [65]:
def main():
    file1 = r"D:\GITHUB\kotak-school-dbms\source_data\Attendance Reports\AttendanceReportUptoSeptember_2024_25.csv"
    file2 = r"D:\GITHUB\kotak-school-dbms\source_data\Attendance Reports\AttendanceOctoberToDecember_2024_25.csv"
    file3 = r"D:\GITHUB\kotak-school-dbms\source_data\Attendance Reports\AttendanceUptoFebruary_2024_25.csv"
    output_file = r"D:\GITHUB\kotak-school-dbms\output_data\AttendanceReport_2024_25.csv"
    
    try:
        print("Loading and cleaning data...\n")
        df = load_and_clean_data(file1, file2, file3)
        print(f"✅ Data loaded with {df.shape[0]} rows.\n")
        
        print("Processing attendance data...\n")
        df_unpivot = process_attendance_data(df)
        df_unpivot.to_csv(output_file, index=False)
        print(f"✅ Processed data with {df_unpivot.shape[0]} rows.\n")
        print("✅ Columns are:\n", df_unpivot.columns)
        print(max(df_unpivot["Date"]))
                
        print("Updating database...\n")
        update_database(df_unpivot)
        print("✅ Data updated successfully!\n")

        print("✅ Attendance report processing completed successfully!\n")
        print(f"✅ No of Rows: {df_unpivot.shape[0]}\n")
        
    except Exception as e:
        print(f"❌ An unexpected error occurred. Error: {e}\n")
        logging.error(f"❌ Unexpected error: {e}\n")


# * Run the script
main()


Loading and cleaning data...

✅ Data loaded with 1780 rows.

Processing attendance data...

✅ Students Before Merging
 1762
✅ Students After Merging
 1733
✅ Processed data with 407255 rows.

✅ Columns are:
 Index(['Date', 'AdmissionNo', 'ClassNo', 'classId', 'branchId',
       'AttendanceStatusId'],
      dtype='object')
2025-03-08 00:00:00
Updating database...

🔄 Truncating table: attendance_report_2024_25
✅ Data copied to 'attendance_report_2024_25' using COPY command!
✅ Data updated successfully!

✅ Attendance report processing completed successfully!

✅ No of Rows: 407255



<h2 align="center"><b>Class Table 2024-25</b></h2>

In [66]:
import pandas as pd
from sqlalchemy import create_engine, text

POSTGRES_CREDENTIALS = {
    "username": "postgres",
    "password": "Hari@123",
    "host": "localhost",
    "port": "5432",
    "database": "schooldb",
}
TABLE_NAME = "class_table_2024_25"

In [67]:
df = pd.read_csv(r"D:\GITHUB\kotak-school-dbms\output_data\class_section_grade_table_2024_25.csv")
df.head()

,ClassNo,Class,classId,className,branchId,branchName
0,1,Pre KG,1,Pre KG,1,Kindergarten
1,2,LKG - A,2,LKG,1,Kindergarten
2,3,LKG - B,2,LKG,1,Kindergarten
3,4,UKG - A,3,UKG,1,Kindergarten
4,5,UKG - B,3,UKG,1,Kindergarten


In [68]:
df.columns

Index(['ClassNo', 'Class', 'classId', 'className', 'branchId', 'branchName'], dtype='object')

In [69]:
import time
import traceback
import logging
import pandas as pd
import urllib
import io
from sqlalchemy import create_engine, text
from sqlalchemy.exc import OperationalError

# Retry settings
MAX_RETRIES = 3
RETRY_DELAY = 5  # Seconds

def bulk_insert_postgres(df, conn, table_name):
    """Fast bulk insert using PostgreSQL COPY command."""
    with conn.connection.cursor() as cur:
        output = io.StringIO()
        df.to_csv(output, sep="\t", index=False, header=False)
        output.seek(0)
        cur.copy_from(output, table_name, sep="\t", null="NULL")
        conn.connection.commit()

def update_database(df):
    """Insert attendance data into PostgreSQL database with retry logic."""
    password = urllib.parse.quote(POSTGRES_CREDENTIALS["password"])
    engine = create_engine(f"postgresql+psycopg2://{POSTGRES_CREDENTIALS['username']}:{password}"
                           f"@{POSTGRES_CREDENTIALS['host']}:{POSTGRES_CREDENTIALS['port']}/{POSTGRES_CREDENTIALS['database']}")

    for attempt in range(1, MAX_RETRIES + 1):
        try:
            print(f"🔄 Attempt {attempt}: Connecting to database {POSTGRES_CREDENTIALS['database']} at {POSTGRES_CREDENTIALS['host']}...")
            with engine.begin() as conn:
                print(f"✅ Connection established.")

                # Create Table if it does not exist
                print(f"Checking if table '{TABLE_NAME}' exists...")
                
                # Truncate the table before inserting data
                print(f"Truncating existing table: {TABLE_NAME}")
                conn.execute(text(f"TRUNCATE TABLE {TABLE_NAME} CASCADE;"))
                
                print(f"Deleting data from {TABLE_NAME} table...")
                conn.execute(text(f"DELETE FROM {TABLE_NAME};"))


                # Fast Bulk Insert
                print(f"Inserting data into {TABLE_NAME} table...")
                bulk_insert_postgres(df, conn, TABLE_NAME)

                print(f"✅ Data successfully inserted into '{TABLE_NAME}' table.")
                return  # Exit function if successful

        except OperationalError as e:
            print(f"❌ OperationalError: {e}")
            logging.error(f"❌ OperationalError: {e}")
            logging.error("Error Traceback:\n" + traceback.format_exc())

            if attempt < MAX_RETRIES:
                print(f"🔄 Retrying in {RETRY_DELAY} seconds...")
                time.sleep(RETRY_DELAY)
            else:
                print("❌ Max retries reached. Could not update the database.")
                logging.error("❌ Max retries reached. Could not update the database.")
                return


In [70]:
update_database(df)

🔄 Attempt 1: Connecting to database schooldb at localhost...
✅ Connection established.
Checking if table 'class_table_2024_25' exists...
Truncating existing table: class_table_2024_25
Deleting data from class_table_2024_25 table...
Inserting data into class_table_2024_25 table...
✅ Data successfully inserted into 'class_table_2024_25' table.


<h2 align="center"><b>FEE COLLECTION REPORT 2024-25</b></h2>

## **Import Required Libraries**

In [196]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib.parse
import numpy as np
import logging
from sqlalchemy import create_engine, text

## **Define Login Credentials and MySQL Credentials*
# * 🔹 Login Credentials
login_url = "https://app.myskoolcom.tech/kotak_vizag/login"
data_url = "https://app.myskoolcom.tech/kotak_vizag/office_fee/fee_consolidate_report_print?&from=2024-04-01&to=&is_transport_fee=&college_id=&course_id=&branch_id=&semister_id=&section_id=&academic_years_id=&payment_type_id=&fee_status=&status=1&imageField=Search"

credentials = {
    "uname": "harikiran",
    "psw": "812551"
}

POSTGRES_CREDENTIALS = {
    "username": "postgres",
    "password": "Hari@123",
    "host": "localhost",
    "port": "5432",
    "database": "schooldb",
}

TABLE_NAME = "fees_collection_2024_25"


In [197]:
## **Function to Log in to Website*
def login_to_website():
    session = requests.Session()
    login_response = session.post(login_url, data=credentials)
    if "Invalid" in login_response.text:
        print("❌ Login failed! Check credentials.\n")
        return None
    else:
        print("✅ Login successful!\n")
        return session

In [198]:
## **Function to Fetch All Fee Tables*
def fetch_all_fee_tables(session):
    response = session.get(data_url)
    soup = BeautifulSoup(response.text, "html.parser")

    # * Find all fee tables
    fee_tables = soup.find_all("table", class_="b-t")

    all_data = []  # * List to store all rows

    # * Loop through each table and extract data
    for table in fee_tables:
        df = table_to_dataframe(table)
        if df is not None:
            all_data.append(df)

    # * Merge all class data into a single DataFrame
    combined_df = pd.concat(all_data, ignore_index=True)
    return combined_df

In [199]:
## **Function to Convert HTML Table to DataFrame*
def table_to_dataframe(table):
    if not table:
        print("❌ No table to convert!")
        return None

    # * Extract column headers
    headers = [th.get_text(strip=True) for th in table.find_all("th")]

    # * Extract table rows
    rows = []
    for tr in table.find_all("tr")[1:]:  # * Skip header row
        cells = [td.get_text(strip=True) for td in tr.find_all("td")]
        if cells:
            rows.append(cells)

    # * Convert to Pandas DataFrame
    df = pd.DataFrame(rows, columns=headers)
    return df

In [200]:


## **Function to Clean Data*
def clean_data(df):
    
    # * Drop rows where the first column starts with "Total"
    df = df[~df.iloc[:, 0].astype(str).str.startswith("Total", na=False)].copy()    
    
    # * Ensure 'Admission No' is treated as a string (no conversion to numeric)
    df.loc[:,'Admin No.'] = df['Admin No.'].astype(str)
    
    df.columns = ['SNo', 'AdmissionNo', 'Name', 'Abacus / Vediic Maths', 'TERM FEE',
       'Total_Fees', 'Abacus / Vediic Maths', 'TERM FEE',
       'Total_Fee_Paid', 'Discount_Concession', 'Total_Due']
    
    # * Convert relevant columns to numeric (removing commas)
    numeric_columns = ["Total_Fees", "Total_Fee_Paid", "Discount_Concession", "Total_Due"]
    
    for col in numeric_columns:
        df[col] = df[col].astype(str)  # * Convert everything to string
        df[col] = df[col].str.replace(",", "")  # * Remove commas
        df[col] = df[col].replace(["", "None", "nan", "NaN", np.nan], np.nan)  # * Replace invalid values with NaN
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)  # * Convert to float, replace NaN with 0
        df[col] = df[col].astype(int)  # * Convert to integer
        
    #df["SNo"] = range(1, len(df) + 1)       
        
    
    df = df.drop(columns=["SNo",'Abacus / Vediic Maths', 'TERM FEE', 'Abacus / Vediic Maths', 'TERM FEE'])
    

    return df

In [ ]:
def update_database(df, TABLE_NAME, POSTGRES_CREDENTIALS):
    """Insert fee collection data into PostgreSQL efficiently, fixing column name issues."""
    
    password = urllib.parse.quote(POSTGRES_CREDENTIALS["password"])
    
    engine = create_engine(
        f"postgresql+psycopg2://{POSTGRES_CREDENTIALS['username']}:{password}"
        f"@{POSTGRES_CREDENTIALS['host']}:{POSTGRES_CREDENTIALS['port']}/{POSTGRES_CREDENTIALS['database']}"
    )

    try:
        with engine.begin() as conn:
            print(f"🔄 Connecting to database {POSTGRES_CREDENTIALS['database']} at {POSTGRES_CREDENTIALS['host']}...")

            # ✅ Fix column names before inserting
            df.columns = df.columns.str.lower()  # Ensure all column names match PostgreSQL
            print("✅ Fixed column names:", df.columns)

            # 🔹 Use DELETE instead of TRUNCATE
            print(f"⚠️ Deleting existing records from: {TABLE_NAME}")
            conn.execute(text(f"DELETE FROM {TABLE_NAME};"))
            print(f"✅ Table '{TABLE_NAME}' cleared.\n")

        # 📥 Insert Data
        print(f"📥 Inserting data into {TABLE_NAME} table...")
        df.to_sql(name=TABLE_NAME, con=engine, if_exists='append', index=False, method='multi', chunksize=1000)
        
        print(f"✅ Data successfully inserted into '{TABLE_NAME}' table.\n")

    except Exception as e:
        print(f"❌ Error occurred while updating database: {e}")
        logging.error(f"Error occurred while updating database: {e}")

    finally:
        engine.dispose()  # Close connection


In [202]:
## **Main Execution Flow*
def main():
    # * Log in to the website
    session = login_to_website()
    if session is None:
        return

    # * Fetch all fee tables
    df = fetch_all_fee_tables(session)

    if not df.empty:
        print("✅ Data extracted successfully! Cleaning data...\n")

        # * Clean the data
        df = clean_data(df)
        print("✅ Columns are:'\n",df.columns)

        # * Save to CSV (optional)
        df.to_csv(r"D:\GITHUB\kotak-school-dbms\output_data\fee_collection_report_2024_25.csv", index=False)
        print("✅ Data saved to 'daywise_fee_collection_report.csv'\n")

        # * Insert data into MySQL database
        update_database(df, TABLE_NAME, POSTGRES_CREDENTIALS)
        print(f"✅ {len(df)} records entered into the database")

        # * Print sample data
        print(df.to_string(index=False))

    else:
        print("❌ No data found! The page structure might have changed.")

In [203]:
## **Run the Main Function*
if __name__ == "__main__":
    main()


✅ Login successful!

✅ Data extracted successfully! Cleaning data...

✅ Columns are:'
 Index(['AdmissionNo', 'Name', 'Total_Fees', 'Total_Fee_Paid',
       'Discount_Concession', 'Total_Due'],
      dtype='object')
✅ Data saved to 'daywise_fee_collection_report.csv'

🔄 Connecting to database schooldb at localhost...
✅ Fixed column names: ['admissionno', 'name', 'total_fees', 'total_fee_paid', 'discount_concession', 'total_due']
⚠️ Deleting existing records from: fees_collection_2024_25
✅ Table 'fees_collection_2024_25' cleared.

📥 Inserting data into fees_collection_2024_25 table...
✅ Data successfully inserted into 'fees_collection_2024_25' table.

✅ 1717 records entered into the database
admissionno                                              name  total_fees  total_fee_paid  discount_concession  total_due
      16922                                   BHASHVIKA AKULA       24200           18150                    0       6050
      17103                            BHAVITHA SRI PANDR

<h2 align="center"><b>FEE CONCESSION REPORT 2024-25</b></h2>

In [204]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib.parse
import urllib.parse
from sqlalchemy import create_engine, MetaData
from sqlalchemy.dialects.postgresql import insert
from datetime import date


## **Define Login Credentials and MySQL Credentials*
login_url = "https://app.myskoolcom.tech/kotak_vizag/login"
data_url = "https://app.myskoolcom.tech/kotak_vizag/office_fee/fee_discounts_report_receipt_wise_print?&academic_years_id=1"

credentials = {
    "uname": "harikiran",
    "psw": "812551"
}

POSTGRES_CREDENTIALS = {
    "username": "postgres",
    "password": "Hari@123",
    "host": "localhost",
    "port": "5432",
    "database": "schooldb",
}

TABLE_NAME = "fee_concession_2024_25"

In [226]:
## **Function to Log in to Website*
def login_to_website():
    session = requests.Session()
    login_response = session.post(login_url, data=credentials)
    
    if login_response.status_code != 200:
        print("❌ Login request failed! Server error.\n")
        return None
    
    soup = BeautifulSoup(login_response.text, "html.parser")
    if soup.find("div", class_="alert-danger"):
        print("❌ Login failed! Check credentials.\n")
        return None
    
    print("✅ Login successful!\n")
    return session

In [227]:
## **Function to Fetch All Fee Tables*
def fetch_all_concession_tables(session):
    response = session.get(data_url)
    soup = BeautifulSoup(response.text, "html.parser")
    
    concession_tables = soup.find_all("table", class_="table_view")
    if not concession_tables:
        print("❌ No fee tables found! The page structure may have changed.")
        return None

    all_data = []
    for table in concession_tables:
        df = table_to_dataframe(table)
        if df is not None:
            all_data.append(df)

    if not all_data:
        print("❌ No data extracted from tables.")
        return None

    combined_df = pd.concat(all_data, ignore_index=True)
    return combined_df

In [228]:
## **Function to Convert HTML Table to DataFrame*
def table_to_dataframe(table):
    headers = [th.get_text(strip=True) for th in table.find_all("th")]
    if len(headers) > 8:
        headers = headers[:8]  # * Keep only the first 8 columns
    
    rows = []
    for tr in table.find_all("tr")[1:]:
        cells = [td.get_text(strip=True) for td in tr.find_all("td")]
        if len(cells) >= 8:
            rows.append(cells[:8])  # * Keep only the first 8 columns
    
    df = pd.DataFrame(rows, columns=headers) if rows else None
    return df

In [229]:
import pandas as pd

def clean_data(df):
    # Standardize column names: lowercase, replace spaces with underscores
    df.columns = df.columns.str.strip().str.replace(" ", "_").str.lower()
    
    # Drop rows where 'student_number' is missing (ensuring data consistency)
    df = df.dropna(subset=["student_number"])
    df["student_number"] = df["student_number"].astype(str).str.strip()

    # Convert 'discount_given' to numeric, handling errors gracefully
    df["discount_given"] = pd.to_numeric(df["discount_given"], errors="coerce").fillna(0.00)

    # Drop extra unnecessary columns if they exist
    df.drop(columns=['receipt_no', 'fee_name', 'fee_amount', 'total_due_amount'], errors="ignore", inplace=True)
    
    df["date"] = df["date"].astype(str).str.strip()  # Remove spaces

    # Ensure 'date' is in correct format & drop invalid entries
    df["date"] = pd.to_datetime(df["date"], errors="coerce").dt.date
    df = df.dropna(subset=["date"])  # Drop rows with invalid dates
    
    # Add an ID column (Auto-incremental)
    df["id"] = range(1, len(df) + 1)

    # Reorder columns for consistency
    df = df[['id', 'date', 'student_number', 'student_name', 'discount_given']]

    # Reset index (ensures DataFrame integrity)
    df.reset_index(drop=True, inplace=True)

    return df


In [230]:
import logging
import urllib.parse
import pandas as pd
from sqlalchemy import create_engine, text

def update_database(df: pd.DataFrame, table_name: str, postgres_credentials: dict):
    """Insert fee collection data into PostgreSQL efficiently, fixing column name issues."""

    # Encode password to handle special characters safely
    password = urllib.parse.quote(postgres_credentials["password"])

    # Create PostgreSQL connection engine
    engine = create_engine(
        f"postgresql+psycopg2://{postgres_credentials['username']}:{password}"
        f"@{postgres_credentials['host']}:{postgres_credentials['port']}/{postgres_credentials['database']}"
    )

    try:
        with engine.begin() as conn:  # Auto-commits or rollbacks if an error occurs
            print(f"🔄 Connecting to database {postgres_credentials['database']} at {postgres_credentials['host']}...")

            # ✅ Standardize column names to lowercase
            df.columns = df.columns.str.lower()
            print("✅ Fixed column names:", df.columns.tolist())

            # 🔹 Use DELETE instead of TRUNCATE for transactional safety
            print(f"⚠️ Deleting existing records from: {table_name}")
            conn.execute(text(f"DELETE FROM {table_name};"))
            print(f"✅ Table '{table_name}' cleared.\n")

        # 📥 Insert Data Efficiently
        print(f"📥 Inserting data into {table_name} table...")
        df.to_sql(name=table_name, con=engine, if_exists="append", index=False, method="multi", chunksize=1000)

        print(f"✅ Data successfully inserted into '{table_name}' table.\n")

    except Exception as e:
        logging.error(f"❌ Error updating database: {e}", exc_info=True)
        print(f"❌ Error occurred while updating database: {e}")

    finally:
        engine.dispose()  # Ensure connection is closed



In [232]:
## **Main Execution Flow*
def main():
    session = login_to_website()
    if session is None:
        return
    
    df = fetch_all_concession_tables(session)
    if df is None or df.empty:
        print("❌ No data found! The page structure might have changed.")
        return
    
    print("✅ Data extracted successfully! Cleaning data...\n")
    df = clean_data(df)
    print("✅ Data cleaned successfully!\n")
    
    print("✅ Columns are:\n", df.columns)
    print(df.dtypes)

    
    output_file = r"D:\\GITHUB\\kotak-school-dbms\\output_data\\fee_concession_report_2024_25.csv"
    df.to_csv(output_file, index=False)
    print(f"✅ Data saved to '{output_file}'\n")
    
    update_database(df, TABLE_NAME, POSTGRES_CREDENTIALS)
    print(f"✅ {len(df)} records entered into the database")
    
    print(df.to_string())

## **Run the Main Function*
if __name__ == "__main__":
    main()

✅ Login successful!

✅ Data extracted successfully! Cleaning data...

✅ Data cleaned successfully!

✅ Columns are:
 Index(['id', 'date', 'student_number', 'student_name', 'discount_given'], dtype='object')
id                  int64
date               object
student_number     object
student_name       object
discount_given    float64
dtype: object
✅ Data saved to 'D:\\GITHUB\\kotak-school-dbms\\output_data\\fee_concession_report_2024_25.csv'

🔄 Connecting to database schooldb at localhost...
✅ Fixed column names: ['id', 'date', 'student_number', 'student_name', 'discount_given']
⚠️ Deleting existing records from: fee_concession_2024_25
✅ Table 'fee_concession_2024_25' cleared.

📥 Inserting data into fee_concession_2024_25 table...
✅ Data successfully inserted into 'fee_concession_2024_25' table.

✅ 90 records entered into the database
    id        date student_number                                   student_name  discount_given
0    1  2024-07-27          15660                   YAAS